<a href="https://colab.research.google.com/github/PALLADIUM26/translaTHOR/blob/main/English_to_Bengali_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Translating English to Bengali

##Installing libraries

In [ ]:
!pip install transformers[torch] -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 21.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylin

In [ ]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [ ]:
!pip install transformers huggingface_hub

## Importing libraries

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.utils.data import DataLoader

In [ ]:
# Load model directly
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-bn-en")
# model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-bn-en")

tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/309M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

##Importing datasets and Preprocess data

In [ ]:
def load_data(english_file, bengali_file):
    with open(english_file, 'r', encoding='utf-8') as f:
        english_lines = f.readlines()
    with open(bengali_file, 'r', encoding='utf-8') as f:
        bengali_lines = f.readlines()

    data = {'translation': [{'en': en.strip(), 'bn': bn.strip()} for en, bn in zip(english_lines, bengali_lines)]}
    return Dataset.from_dict(data)

# Assuming 'train.en.txt' and 'train.bn.txt' are your training files
train_dataset = load_data('suparadev_en.txt', 'suparadev_bn.txt')
test_dataset = load_data('suparatest2018_en.txt', 'suparatest2018_bn.txt')

In [ ]:
# Check the structure of the dataset
print(train_dataset)
print(test_dataset)

Dataset({
    features: ['translation'],
    num_rows: 500
})
Dataset({
    features: ['translation'],
    num_rows: 500
})


## Tokenize the Data

In [ ]:
# This one is publically available.
tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-bn-en")
# tokenizer = AutoTokenizer.from_pretrained('facebook/nllb-200-distilled-600M')

# Set the target language for the tokenizer
tokenizer.tgt_lang = 'ben' # Set to Bengali

def preprocess_function(examples):
    inputs = [ex['en'] for ex in examples['translation']]
    targets = [ex['bn'] for ex in examples['translation']]
    # model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True, padding='max_length')
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True, padding=True)
    model_inputs = {k: [v for v in values if v is not None] for k, values in model_inputs.items()}
    return model_inputs

# train_dataset = train_dataset.map(preprocess_function, batched=True)
# test_dataset = test_dataset.map(preprocess_function, batched=True)

train_dataset.set_transform(preprocess_function)
test_dataset.set_transform(preprocess_function)

train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=1)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.12M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/806k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.25M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


## Load the Model

In [ ]:
# model = AutoModelForSeq2SeqLM.from_pretrained('facebook/nllb-200-distilled-600M')
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-bn-en")

pytorch_model.bin:   0%|          | 0.00/309M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

## Train the Model

In [ ]:
# !pip install accelerate -U

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,  # Enable mixed precision training
    gradient_accumulation_steps=4,  # Accumulate gradients
    remove_unused_columns=False # Tell the trainer to remove unused columns.
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,4.549706
2,No log,4.074535
3,No log,3.934892


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[63596]], 'forced_eos_token_id': 0}


TrainOutput(global_step=24, training_loss=3.537225087483724, metrics={'train_runtime': 1704.1972, 'train_samples_per_second': 0.88, 'train_steps_per_second': 0.014, 'total_flos': 46240547733504.0, 'train_loss': 3.537225087483724, 'epoch': 3.0})

In [ ]:
# def translate_text(input_text):
#     # Tokenize the input text
#     inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True, padding="max_length")

#     # Generate translation using the model
#     outputs = model.generate(
#         input_ids=inputs["input_ids"],
#         attention_mask=inputs["attention_mask"],
#         max_length=128,
#         num_beams=5,
#         num_return_sequences=1,
#         early_stopping=True,
#         temperature=0.7,
#         top_k=50,
#         top_p=0.95
#     )

#     # Decode the generated text
#     translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
#     return translated_text

# # Get user input and generate translation
# user_input = input("Enter the text to be translated from English to Bengali: ")
# translated_output = translate_text(user_input)
# print("Translated text:", translated_output)

import logging
import transformers
logging.basicConfig(level=logging.INFO)
transformers.logging.set_verbosity_info()


def translate_text(input_text):
    logging.info(f"Input text: {input_text}")

    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True, padding="max_length")
    logging.info(f"Tokenized input: {inputs}")

    # Generate translation using the model
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=128,
        num_beams=5,
        num_return_sequences=1,
        early_stopping=True,
        temperature=0.7,
        top_k=50,
        top_p=0.95
    )
    logging.info(f"Generated outputs: {outputs}")

    # Decode the generated text
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    logging.info(f"Translated text: {translated_text}")

    return translated_text

# Get user input and generate translation
user_input = input("Enter the text to be translated from English to Bengali: ")
translated_output = translate_text(user_input)
print("Translated text:", translated_output)


Enter the text to be translated from English to Bengali: what you do


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:540: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:545: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Translated text: io y


## Evaluate the Model

In [ ]:
metrics = trainer.evaluate()
print(metrics)

{'eval_loss': 1.886110782623291, 'eval_runtime': 137.6654, 'eval_samples_per_second': 3.632, 'eval_steps_per_second': 3.632, 'epoch': 3.0}


## Save the Model

In [ ]:
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[63596]], 'forced_eos_token_id': 0}


('./saved_model/tokenizer_config.json',
 './saved_model/special_tokens_map.json',
 './saved_model/vocab.json',
 './saved_model/source.spm',
 './saved_model/target.spm',
 './saved_model/added_tokens.json')

## Output for User input

In [ ]:
# Load the saved model and tokenizer
model_dir = "./saved_model"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

def translate_text(input_text):
    # Tokenize the input text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=128, truncation=True)

    # Generate translation using the model
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)

    # Decode the generated text
    translated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return translated_text

# Get user input and generate translation
user_input = input("Enter the text to be translated from English to Bengali: ")
print(user_input)
translated_output = translate_text(user_input)
print("Translated text:", translated_output)

Enter the text to be translated from English to Bengali: he
he
Translated text: সে


## Convert the saved folder into zip

In [ ]:
import zipfile
import os
from google.colab import files

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, os.path.relpath(file_path, folder_path))

# Path to the folder you want to zip
folder_path = './saved_model'

# Path for the output zip file
output_filename = 'final0.zip'

# Create a zip file
zip_folder(folder_path, output_filename)

# Download the zip file
# files.download(output_filename)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>